In [33]:
from pyspark.sql import SparkSession
from datetime import datetime

if spark is not None:
    spark.stop()
spark = SparkSession \
        .builder \
        .getOrCreate()
spark


25/02/05 11:58:59 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


# W5M1
우선 hdfs에 파일을 업로드 합니다.

In [17]:
!pwd
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet
!hdfs dfs -mkdir /yellow_texi
!hdfs dfs -put ./yellow_tripdata_2024-03.parquet /yellow_texi/2024-03.parquet
!rm ./yellow_tripdata_2024-03.parquet

/usr
--2025-02-04 16:03:56--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.138.245.16, 108.138.245.225, 108.138.245.96, ...
connected. to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.138.245.16|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 60078280 (57M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-03.parquet’

yellow_tripdata_202 100%[===================>]  57.29M  11.0MB/s    in 6.0s    

2025-02-04 16:04:02 (9.62 MB/s) - ‘yellow_tripdata_2024-03.parquet’ saved [60078280/60078280]

Picked up _JAVA_OPTIONS: -Xms512m -Xmx8g
2025-02-04 16:04:03,357 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
mkdir: `/yellow_texi': File exists
Picked up _JAVA_OPTIONS: -Xms512m -Xmx8g
2025-02-04 16:04:04,319 WARN util.NativeCodeLoader: Unable to load native-had

In [34]:
df = spark.read.parquet("hdfs://master:9000/yellow_texi/2024-03.parquet")
rdd = df.rdd
df.columns
df.dtypes

[('VendorID', 'int'),
 ('tpep_pickup_datetime', 'timestamp_ntz'),
 ('tpep_dropoff_datetime', 'timestamp_ntz'),
 ('passenger_count', 'bigint'),
 ('trip_distance', 'double'),
 ('RatecodeID', 'bigint'),
 ('store_and_fwd_flag', 'string'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('payment_type', 'bigint'),
 ('fare_amount', 'double'),
 ('extra', 'double'),
 ('mta_tax', 'double'),
 ('tip_amount', 'double'),
 ('tolls_amount', 'double'),
 ('improvement_surcharge', 'double'),
 ('total_amount', 'double'),
 ('congestion_surcharge', 'double'),
 ('Airport_fee', 'double')]

## Data Cleaning
다음으로는 결측치가 있는 행을 제거합니다.

In [35]:
rdd_no_null = rdd.filter(lambda row: all(value is not None for value in row))

## Transformation Logic
다음으로는 Transformation을 수행합니다.
* 이동하는데 0원이나 음수인 금액을 지불한 행은 제거
* MAP & Reduce: 총 수익과 총 여행횟수를 계산
* 날짜별로 그룹화 하여 일일 지표를 계산

우선 날짜별로 groupby를 한 뒤, 일일 지표를 계산하고, 일일지표를 다시 모아 reduce해서 총 수익과 총 여행횟수를 계산한다.


In [36]:
# 금액이 0원 이하인 행을 필터링
rdd_filtered = rdd_no_null.filter(lambda row: row['total_amount'] > 0)

# (날짜, 원하는 데이터) 형식으로 변환
count_mapped = rdd_filtered.map(lambda row: (row['tpep_dropoff_datetime'].date(), 1))
revenue_mapped = rdd_filtered.map(lambda row: (row['tpep_dropoff_datetime'].date(), row['total_amount']))
distance_mapped = rdd_filtered.map(lambda row: (row['tpep_dropoff_datetime'].date(), row['trip_distance']))

# get daily trip count and total amount
daily_count = count_mapped.reduceByKey(lambda x, y: x + y).sortByKey()
daily_revenue = revenue_mapped.reduceByKey(lambda x, y: x + y).sortByKey()





## Aggregation Logic
다음을 계산하고 출력합니다:
* 총 운송 수
* 운송으로 인한 이익 총합
* 평균 운행 거리
* 일간 평균 운송 수
* 일간 평균 이익

In [37]:
_, trip_count = count_mapped.reduce(lambda x, y: (0, x[1] + y[1]))
_, total_amount = revenue_mapped.reduce(lambda x, y: (0, x[1] + y[1]))
_, total_trip_dist = distance_mapped.reduce(lambda x, y: (0, x[1] + y[1]))
avg_trip_dist = total_trip_dist / trip_count
daily_count_list = daily_count.collect()
daily_revenue_list = daily_revenue.collect()

print("total number of trips: ", trip_count)
print("total revenue generated from the trips: ", total_amount)
print("average trip distance", avg_trip_dist)

print("\ntrips per day, average:", trip_count / 30)
for row in daily_count_list:
    print(row[0], ":", row[1])

print("\ntotal revenue per day, average:", total_amount / 30)
for row in daily_revenue_list:
    print(row[0], ":", row[1])




total number of trips:  3111733
total revenue generated from the trips:  88379924.18011408
average trip distance 3.5320004190589693

trips per day, average: 103724.43333333333
2002-12-31 : 1
2003-01-01 : 1
2024-02-29 : 3
2024-03-01 : 105907
2024-03-02 : 104370
2024-03-03 : 87344
2024-03-04 : 88873
2024-03-05 : 103073
2024-03-06 : 111461
2024-03-07 : 109783
2024-03-08 : 106869
2024-03-09 : 111700
2024-03-10 : 88513
2024-03-11 : 90463
2024-03-12 : 101398
2024-03-13 : 108226
2024-03-14 : 117946
2024-03-15 : 111582
2024-03-16 : 109949
2024-03-17 : 88955
2024-03-18 : 87868
2024-03-19 : 101086
2024-03-20 : 106045
2024-03-21 : 112693
2024-03-22 : 101946
2024-03-23 : 97287
2024-03-24 : 85914
2024-03-25 : 85013
2024-03-26 : 99088
2024-03-27 : 105486
2024-03-28 : 106968
2024-03-29 : 99464
2024-03-30 : 94313
2024-03-31 : 81546
2024-04-01 : 597
2024-04-02 : 2

total revenue per day, average: 2945997.472670469
2002-12-31 : 18.0
2003-01-01 : 25.15
2024-02-29 : 60.91
2024-03-01 : 2917914.009999908
20